In [61]:
# Reduce MIRI imaging data with JWST pipeline
import numpy as np
import os
from astropy.io import fits
from glob import glob
import multiprocessing

# Set the CRDS path and url
# should modify the path according to your computer!!
# This must be set before importing the pipeline!
# Note the PUB has been decommissioned recently, so we use OPS 
os.environ['CRDS_PATH']= '/Users/users/gyang/gyang/crds_cache/jwst_ops'
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

import jwst
from jwst import pipeline
from jwst.pipeline import Detector1Pipeline
from jwst.pipeline import Image2Pipeline
from jwst.pipeline import Image3Pipeline
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3 import Asn_Lv3Image
from jwst import datamodels
from jwst.resample.resample_utils import build_driz_weight

from astropy.table import Table
from astropy.wcs import WCS
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.stats import sigma_clipped_stats
import astropy.units as u

from scipy.stats import median_abs_deviation as mad

import matplotlib.pyplot as plt

%run -i 'write_rms_map.ipynb'
%run -i 'run_tkreg.ipynb'
%run -i 'super_bg.ipynb'
%run -i 'group_imgs.ipynb'
%run -i 'correct_jump.ipynb'

# Should modify the paths below according to your computer!!
# Set the input directory for all raw data
# For June dataset
#input_dirs = glob("../data/raw/jw01345/mastDownload/JWST/*mirimage/"); dec_flag = False
# For December dataset
input_dirs = glob("../data/raw/jw01345_dec/mastDownload/JWST/*mirimage/"); dec_flag = True
# For both datasets
# Note: this can only be used for stage 1 & 2 (1st cell below)
#input_dirs = glob("../data/raw/jw01345*/mastDownload/JWST/*mirimage/")

# Set the output directory for reduced files
#output_dir = "/Users/users/gyang/sterm_bkup/large_data/projects/ceers/ero/my_reduced/"
output_dir = "../data/reduced/"

In [62]:
# re-run flags: if False, will not rerun the pipeline if file output file exists
rerun_stage1 = False
rerun_stage2 = False

## Stage 1
# The list of raw files that have the jump issue
jump_lists = []
for i in [1, 2]:
    for j in [1, 2, 3]:
        jump_lists.append( 'jw0134500%d001_14101_0000%d_mirimage_uncal.fits' %(i,j) )

# List to save stage 1 result file
stage1_files = []
# Iterate over each input directory 
for input_dir in input_dirs:
    # Search for raw files in the directory
    raw_files = sorted(glob(input_dir+'*_uncal.fits'))
    # Iterate over each raw file
    for file in raw_files:
        # Check if the output file exists 
        stage1_files.append( output_dir + raw_files[0].split('/')[-1].replace('uncal', 'rate') )
        # Skip stage 1 if result exsits and rerun_flag=False
        if (len(glob(stage1_files[-1]))==1) and (not rerun_stage1):
            print(stage1_files[-1], 'exists; skip...')
            continue
        # Set up the pipeline
        det1 = Detector1Pipeline()
        # Use parallel, can be 'none', 'quarter', 'half', and 'all', or a fractional number
        det1.ramp_fit.maximum_cores = 'quarter'
        det1.jump.maximum_cores = det1.ramp_fit.maximum_cores
        
        # Treat the jump exposures separately 
        if file.split('/')[-1] in jump_lists: 
            # Run till jump_fit 
            # (i.e., skip the last two steps)
            det1.ramp_fit.skip = True
            det1.gain_scale.skip = True
            res = det1.run(file)
            # Do the jump correction
            res = correct_jump(res)
            # Run ramp_fit
            # (gain_scale is skipped by default, so don't have to run it)
            det1.ramp_fit.skip = False
            det1.ramp_fit.save_results = True
            res = det1.ramp_fit(res)
            # Rename and move the products 
            rate_fname = glob('*0_ramp_fit.fits')[0]
            rateints_fname = glob('*1_ramp_fit.fits')[0]
            os.system('mv ' + rate_fname     + ' ' + output_dir + \
                       rate_fname.replace('0_ramp_fit', 'rate'))
            os.system('mv ' + rateints_fname + ' ' + output_dir + \
                       rateints_fname.replace('1_ramp_fit', 'rateints'))
        else:
            # Run the stage 1 pipeline
            det1.save_results = True
            det1.run(file)
            # Move the result to output directory 
            os.system('mv *.fits ' + output_dir)

            
## Stage 2  
# List to save stage 2 result file
stage2_files = []
# Search for stage-1 result files
#stage1_files = sorted(glob(output_dir+'*_rate.fits'))
# Group stage 1 file by band
flt_groups, flt_vals = group_imgs(stage1_files, 'band')
# Iterate over each file
for files, flt_val in zip(flt_groups, flt_vals):
    for file in files:
        # Check if the output file exists 
        stage2_files.append( file.replace('rate', 'cal') )
        # Skip stage 1 if result exsits and rerun_flag=False
        if (len(glob(stage2_files[-1]))==1) and (not rerun_stage2):
            print(stage2_files[-1], 'exists; skip...')
            continue
        # Set up the pipeline
        img2 = Image2Pipeline()
        img2.save_results = True
        # For F560W & F770W, use FLIGHT/FAST flat instead of the default GROUND/SLOW flat
        if   flt_val=='F560W': 
            img2.flat_field.user_supplied_flat = os.environ['CRDS_PATH'] + \
                '/references/jwst/miri/jwst_miri_flat_0787.fits'
        elif flt_val=='F770W':
            img2.flat_field.user_supplied_flat = os.environ['CRDS_PATH'] + \
                '/references/jwst/miri/jwst_miri_flat_0785.fits'
        # Run the pipeline
        img2.run(file)
        # Move the result to output directory 
        os.system('mv *.fits ' + output_dir)

../data/reduced/jw01345016001_07201_00004_mirimage_rate.fits exists; skip...
../data/reduced/jw01345013002_02201_00001_mirimage_rate.fits exists; skip...
../data/reduced/jw01345012001_05201_00001_mirimage_rate.fits exists; skip...
../data/reduced/jw01345015001_02201_00001_mirimage_rate.fits exists; skip...
../data/reduced/jw01345015001_07201_00002_mirimage_rate.fits exists; skip...
../data/reduced/jw01345016001_05201_00001_mirimage_rate.fits exists; skip...
../data/reduced/jw01345012001_07201_00004_mirimage_rate.fits exists; skip...
../data/reduced/jw01345016001_08201_00001_mirimage_rate.fits exists; skip...
../data/reduced/jw01345013001_02201_00002_mirimage_rate.fits exists; skip...
../data/reduced/jw01345016001_07201_00003_mirimage_rate.fits exists; skip...
../data/reduced/jw01345016001_03201_00001_mirimage_rate.fits exists; skip...
../data/reduced/jw01345012001_03201_00001_mirimage_rate.fits exists; skip...
../data/reduced/jw01345012001_07201_00003_mirimage_rate.fits exists; skip...

In [63]:
# Subtract background for stage-2 result 
# Using Casey's algorithm

# re-run flags: if False, will not rerun the pipeline if file output file exists
rerun_stprm = True
rerun_bgsub = False

# List to save strip (median kg) removal result file
stage2_stprm_files = []
# Remove strips or global background 
# Parallel version
#Nc = min(multiprocessing.cpu_count()//4, len(stage2_files))
#with multiprocessing.Pool(processes=Nc) as pool_obj:
#    pool_obj.map(rm_strip, stage2_files)
# Normal version
# Initialize a list to save global background
glob_bkgs = []
for file in stage2_files:
    # Check if the output file exists 
    stage2_stprm_files.append( file.replace('.fits', '_stprm.fits') )
    # Skip if result exsits and rerun_flag=False
    if (len(glob(stage2_stprm_files[-1]))==1) and (not rerun_stprm):
        print(stage2_stprm_files[-1], 'exists; skip...')
        continue
    # Run the strip or median-bkg removal 
    glob_bkgs.append( rm_strip(file, rm_hz=True, rm_vt=True) )
    
# Remove background 
# List to save background removal result file
stage2_bgsub_files = []
# Do a initial group 
init_groups, init_vals = group_imgs(stage2_stprm_files, 'field_band')
# Flag (if True) to indicate the group/val has been used
init_flags = [False]*len(init_vals)
# The list to save final group/val
final_groups, final_vals = [], []

# Iterate over the initial group/val
for idx, group in enumerate(init_groups):
    val = init_vals[idx]
    # Check if this group has been used
    if init_flags[idx]: continue
    # If number of files >=4, accept it in the final list
    if len(group)>=4: 
        # Save to final
        final_vals.append(val)
        final_groups.append(group)
        init_flags[idx] = True
    else:
        # Merge all fields of the same band
        field = val.split('_')[0]
        band = val.split('_')[-1]
        # Find the other same field 
        for oth_idx in range(idx+1, len(init_vals)):
            if (init_vals[oth_idx].split('_')[-1]==band) and \
               (len(init_groups[oth_idx])==3): break
        # Save to final 
        final_vals.append(band)
        final_groups.append( group + init_groups[oth_idx] )
        init_flags[idx] = True
        init_flags[oth_idx] = True

# A function to merge groups
def merge_group(new_val, old_valA, old_valB):
    # Check if new_val already exists
    if new_val in final_vals:
        # Search for it and append the groups from old_val
        idx = final_vals.index(new_val)
        final_groups[idx] = final_groups[idx] + \
                                    final_groups[final_vals.index(old_valA)] + \
                                    final_groups[final_vals.index(old_valB)]
    else:
        # Append the new name
        final_vals.append(new_val)
        final_groups.append( final_groups[final_vals.index(old_valA)] + \
                             final_groups[final_vals.index(old_valB)])
    # Remove the orignal MIRI3 and MIRI6
    rm_idx = final_vals.index(old_valA)
    final_vals.remove( final_vals[rm_idx] )
    final_groups.remove( final_groups[rm_idx] )
    rm_idx = final_vals.index(old_valB)
    final_vals.remove( final_vals[rm_idx] )
    final_groups.remove( final_groups[rm_idx] )   
    return
        
# For the December dataset
# separate the two problematic files from the large set
if dec_flag:
    f1500w_idx = final_vals.index('F1500W')
    bad_files = ['../data/reduced/jw01345012002_02201_00001_mirimage_cal_stprm.fits',
                 '../data/reduced/jw01345012003_02201_00001_mirimage_cal_stprm.fits']
    for bad_file in bad_files:
        final_groups[f1500w_idx].remove(bad_file)
    # Create a new group
    final_vals.append( 'F1500W_bad' )
    final_groups.append( bad_files )
    
    # Manually merge all F560W & F1800W, as their result doesn't look good
    merge_group('F560W',  'MIRI7_F560W',  'MIRI9_F560W')
    merge_group('F1800W', 'MIRI5_F1800W', 'MIRI8_F1800W')
    
else:
    # For the June set
    # Manually regroup the F2100W dithers according to their bkg pattern 
    # First remove the old F2100W groups
    rm_idx = final_vals.index('MIRI1_F2100W')
    final_vals.remove( final_vals[rm_idx] )
    final_groups.remove( final_groups[rm_idx] )
    rm_idx = final_vals.index('MIRI2_F2100W')
    final_vals.remove( final_vals[rm_idx] )
    final_groups.remove( final_groups[rm_idx] ) 
    # Then manually add the groups 
    final_vals.append('F2100Wa')
    final_groups.append( glob('../data/reduced/jw0134500*001_12101_0000*_mirimage_cal_stprm.fits') )
    final_vals.append('F2100Wb')
    final_groups.append( glob('../data/reduced/jw0134500*001_14101_0000*_mirimage_cal_stprm.fits') + \
                         glob('../data/reduced/jw01345052001_02101_0000*_mirimage_cal_stprm.fits') )
    
    # Manually merge all F770W, as their result doesn't look good
    merge_group('F770W',  'MIRI3_F770W',  'MIRI6_F770W')

# subtract the background group by group
for flt, files in zip(final_vals, final_groups):
    #if 'F1800W' not in flt: continue
    # Flag to indicate all files of this band exist
    all_exist = True
    # Iterate over each file to see if exist
    for file in files:
        stage2_bgsub_files.append( file.replace('.fits', '_bgsub.fits') )
        if len(glob(stage2_bgsub_files[-1]))!=1: 
            all_exist=False
    # Skip if result exsits and rerun_flag=False
    print(flt+':')
    if all_exist and (not rerun_bgsub):
        print('file exists and skip...')
        continue
    # Write background for MIRI5/F1800W to check the galaxy group
    #if flt in ['MIRI5_F1800W']: write_bg=True
    #else: write_bg=False
    write_bg = False
    # Run super background
    super_bg(files, write_bg=write_bg)

# Build MIRI5 F1800W background entirely based on MIRI8 F1800W
# to avoid background over-subtraction near the z=1.85 group  
#miri8_f1800w_idx = final_vals.index('MIRI8_F1800W')
#miri5_f1800w_idx = final_vals.index('MIRI5_F1800W')
#for file_idx in range(4):
#    buf_groups = [ final_groups[miri5_f1800w_idx][file_idx] ]
#    buf_groups.extend( final_groups[miri8_f1800w_idx] )
#    super_bg(buf_groups)

2023-07-09 12:58:42,401 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
2023-07-09 12:58:42,402 - stpipe - WARNING -   mean = np.nanmean(data_clipped, axis=axis)
2023-07-09 12:58:42,402 - stpipe - WARNING - 
2023-07-09 12:58:42,403 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
2023-07-09 12:58:42,404 - stpipe - WARNING -   result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
2023-07-09 12:58:42,404 - stpipe - WARNING - 
2023-07-09 12:58:42,473 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
2023-07-09 12:58:42,474 - stpipe - WARNING -   var = nanvar(a, axis=axis, d

2023-07-09 12:58:48,797 - stpipe - WARNING - 
2023-07-09 12:58:48,869 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
2023-07-09 12:58:48,870 - stpipe - WARNING -   var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
2023-07-09 12:58:48,871 - stpipe - WARNING - 
2023-07-09 12:58:49,648 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
2023-07-09 12:58:49,649 - stpipe - WARNING -   mean = np.nanmean(data_clipped, axis=axis)
2023-07-09 12:58:49,649 - stpipe - WARNING - 
2023-07-09 12:58:49,650 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
2023-07-09 12:58:49,651 - stpipe - WAR

2023-07-09 12:58:55,897 - stpipe - WARNING - 
2023-07-09 12:58:55,898 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
2023-07-09 12:58:55,899 - stpipe - WARNING -   result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
2023-07-09 12:58:55,899 - stpipe - WARNING - 
2023-07-09 12:58:55,962 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
2023-07-09 12:58:55,963 - stpipe - WARNING -   var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
2023-07-09 12:58:55,963 - stpipe - WARNING - 
2023-07-09 12:58:56,703 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
2023-07-09 1

2023-07-09 12:59:02,157 - stpipe - WARNING - 
2023-07-09 12:59:02,901 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
2023-07-09 12:59:02,902 - stpipe - WARNING -   mean = np.nanmean(data_clipped, axis=axis)
2023-07-09 12:59:02,903 - stpipe - WARNING - 
2023-07-09 12:59:02,903 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
2023-07-09 12:59:02,904 - stpipe - WARNING -   result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
2023-07-09 12:59:02,904 - stpipe - WARNING - 
2023-07-09 12:59:02,972 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
2023-07-09 12:59:02,973 - stp

2023-07-09 12:59:09,107 - stpipe - WARNING - 
2023-07-09 12:59:09,177 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
2023-07-09 12:59:09,178 - stpipe - WARNING -   var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
2023-07-09 12:59:09,178 - stpipe - WARNING - 
2023-07-09 12:59:09,945 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
2023-07-09 12:59:09,946 - stpipe - WARNING -   mean = np.nanmean(data_clipped, axis=axis)
2023-07-09 12:59:09,947 - stpipe - WARNING - 
2023-07-09 12:59:09,947 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
2023-07-09 12:59:09,948 - stpipe - WAR

2023-07-09 12:59:15,750 - stpipe - WARNING - 
2023-07-09 12:59:15,751 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
2023-07-09 12:59:15,751 - stpipe - WARNING -   result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
2023-07-09 12:59:15,752 - stpipe - WARNING - 
2023-07-09 12:59:15,822 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
2023-07-09 12:59:15,822 - stpipe - WARNING -   var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
2023-07-09 12:59:15,823 - stpipe - WARNING - 
2023-07-09 12:59:16,615 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
2023-07-09 1

2023-07-09 12:59:22,107 - stpipe - WARNING - 
2023-07-09 12:59:22,875 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
2023-07-09 12:59:22,876 - stpipe - WARNING -   mean = np.nanmean(data_clipped, axis=axis)
2023-07-09 12:59:22,876 - stpipe - WARNING - 
2023-07-09 12:59:22,877 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
2023-07-09 12:59:22,878 - stpipe - WARNING -   result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
2023-07-09 12:59:22,878 - stpipe - WARNING - 
2023-07-09 12:59:22,945 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
2023-07-09 12:59:22,946 - stp

2023-07-09 12:59:29,088 - stpipe - WARNING - 
2023-07-09 12:59:29,155 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
2023-07-09 12:59:29,156 - stpipe - WARNING -   var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
2023-07-09 12:59:29,156 - stpipe - WARNING - 
2023-07-09 12:59:29,919 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/astropy/stats/sigma_clipping.py:1030: RuntimeWarning: Mean of empty slice
2023-07-09 12:59:29,920 - stpipe - WARNING -   mean = np.nanmean(data_clipped, axis=axis)
2023-07-09 12:59:29,921 - stpipe - WARNING - 
2023-07-09 12:59:29,922 - stpipe - WARNING - /net/dataserver3/data/users/gyang/conda_env/jwst_pipeline/lib/python3.11/site-packages/numpy/lib/nanfunctions.py:1095: RuntimeWarning: All-NaN slice encountered
2023-07-09 12:59:29,923 - stpipe - WAR

MIRI7_F770W:
file exists and skip...
MIRI5_F1000W:
file exists and skip...
MIRI9_F770W:
file exists and skip...
F1500W:
file exists and skip...
MIRI8_F1000W:
file exists and skip...
F1280W:
file exists and skip...
F1500W_bad:
file exists and skip...
F560W:
file exists and skip...
F1800W:
file exists and skip...


In [64]:
# ETC background value
etc_bkgs = {\
    'MIRI1_F770W':6.73, 'MIRI1_F1000W':16.96, 'MIRI1_F1280W':32.58, 'MIRI1_F1500W':50.25, 'MIRI1_F1800W':97.68, 'MIRI1_F2100W':200.75,
    'MIRI2_F770W':6.73, 'MIRI2_F1000W':16.96, 'MIRI2_F1280W':32.58, 'MIRI2_F1500W':50.25, 'MIRI2_F1800W':97.68, 'MIRI2_F2100W':200.75,
    'MIRI3_F560W':1.24, 'MIRI3_F770W':6.73,
    'MIRI6_F560W':1.24, 'MIRI6_F770W':6.73,
    'MIRI5_F1000W':16.86, 'MIRI5_F1280W':32.63, 'MIRI5_F1500W':50.41, 'MIRI5_F1800W':97.98,
    'MIRI8_F1000W':16.86, 'MIRI8_F1280W':32.63, 'MIRI8_F1500W':50.41, 'MIRI8_F1800W':97.98,
    'MIRI7_F560W':1.30, 'MIRI7_F770W':6.60,
    'MIRI9_F560W':1.30, 'MIRI9_F770W':6.60}
            
# Get the median global background per field_band
measured_bkgs = {}
for group, val in zip(init_groups, init_vals):
    # Print the band_field
    print(val, 'ETC bkg=%5.1f MJy/sr' %etc_bkgs[val])
    # Iterate over each file
    group_bkgs = []
    for file in group:
        # Search the file index 
        idx = stage2_stprm_files.index(file)
        # Print the file name
        print(stage2_stprm_files[idx].split('/')[-1], end='; ')
        # Print the background value
        print('bkg=%5.1f' %glob_bkgs[idx])
        # Save the result
        group_bkgs.append(glob_bkgs[idx])
    # Get the median
    measured_bkgs[val] = np.median(group_bkgs)
    print('measured median = %5.1f' %np.median(group_bkgs))
    print('')
    
# Save to file 
# Convert to array
field_bands, measured_bkg_arrs, etc_bkg_arrs = [], [], []
# Itereate over each field_band
for field_band in measured_bkgs:
    # Fill each field-band value to the array
    field_bands.append(field_band)
    measured_bkg_arrs.append( measured_bkgs[field_band] )
    etc_bkg_arrs.append( etc_bkgs[field_band] )
# Save to npy file
arr_to_save = np.array( [field_bands, measured_bkg_arrs, etc_bkg_arrs], dtype=object)
if dec_flag:
    np.save('../data/bkg/dec_bkg_measured_etc.npy', arr_to_save)
else:
    np.save('../data/bkg/june_bkg_measured_etc.npy', arr_to_save)

MIRI7_F560W ETC bkg=  1.3 MJy/sr
jw01345016001_03201_00001_mirimage_cal_stprm.fits; bkg=  1.8
jw01345016001_02201_00001_mirimage_cal_stprm.fits; bkg=  1.9
jw01345016001_02201_00002_mirimage_cal_stprm.fits; bkg=  1.9
jw01345016001_02201_00003_mirimage_cal_stprm.fits; bkg=  1.8
jw01345016001_02201_00004_mirimage_cal_stprm.fits; bkg=  1.8
measured median =   1.8

MIRI7_F770W ETC bkg=  6.6 MJy/sr
jw01345016001_07201_00004_mirimage_cal_stprm.fits; bkg=  6.1
jw01345016001_05201_00001_mirimage_cal_stprm.fits; bkg=  6.1
jw01345016001_08201_00001_mirimage_cal_stprm.fits; bkg=  6.1
jw01345016001_07201_00003_mirimage_cal_stprm.fits; bkg=  6.1
jw01345016001_07201_00002_mirimage_cal_stprm.fits; bkg=  6.1
jw01345016001_07201_00001_mirimage_cal_stprm.fits; bkg=  6.1
jw01345016002_02201_00001_mirimage_cal_stprm.fits; bkg=  6.1
jw01345016001_05201_00002_mirimage_cal_stprm.fits; bkg=  6.1
jw01345016003_02201_00001_mirimage_cal_stprm.fits; bkg=  6.1
measured median =   6.1

MIRI9_F560W ETC bkg=  1.3 MJy/

In [58]:
etc_bkg_arrs

[6.73,
 6.73,
 50.25,
 1.24,
 1.24,
 6.73,
 200.75,
 50.25,
 16.96,
 200.75,
 97.68,
 16.96,
 6.73,
 32.58,
 32.58,
 97.68]

In [5]:
## Stage 3
# Set version number
version = 'v0.3'

# Set the output shape and tangential point offset
out_pars = {'MIRI1':((1500,1500), (60.5, 60.5)), 'MIRI2':((1500,1500), (60.5, 60.5)),
            'MIRI3':((1500,1500), (60.5, 60.5)), 'MIRI6':((1500,1500), (60.5, 60.5)),
            'MIRI5':((1800,2000),(150.5, 60.5)), 'MIRI7':((1800,2000), (60.5, 60.5)),
            'MIRI8':((1800,2000),(150.5, 60.5)), 'MIRI9':((1800,2000), (60.5, 60.5))}

# Group files by field_filter
ff_groups, ff_vals = group_imgs(stage2_bgsub_files, 'field_band')

# Determine the tangential x,y for each field
tg_xys = {'MIRI1':None, 'MIRI2':None, 'MIRI3':None, 'MIRI6':None,\
          'MIRI5':None, 'MIRI7':None, 'MIRI8':None, 'MIRI9':None}
for field in tg_xys.keys():
    # Check if the field exists in this run
    try:
        group = ff_groups[ ff_vals.index(field+'_F770W') ]
    except:
        try:
            group = ff_groups[ ff_vals.index(field+'_F1500W') ]
        except:
            print(field, 'does not exist; skip...')
            continue
    
    #if field not in ['MIRI5']: continue
    # prepare stuff for the resample run        
    asn = dict( asn_from_list.asn_from_list(group, rule=Asn_Lv3Image, 
                product_name='l3_results', asn_type="image3") )
    img3 = Image3Pipeline()
    # First resample to get the coordinate transformation
    img3.resample.rotation = -49.7
    img3.resample.pixel_scale = 0.09
    img3.resample.output_shape = out_pars[field][0]
    tg_ra, tg_dec = 214.825, 52.825
    img3.resample.crval = tg_ra, tg_dec
    res = img3.resample(asn)    
    tg_x, tg_y = res.get_fits_wcs().world_to_pixel( SkyCoord(tg_ra*u.degree, tg_dec*u.degree, frame='icrs') )
    # Round to 0.5 to align with HST 
    tg_xys[field] = np.round(tg_x)+out_pars[field][1][0],\
                    np.round(tg_y)+out_pars[field][1][1]
    
for ff_idx in range(len(ff_vals)):
    # Get the field and group info
    field = ff_vals[ff_idx].split('_')[0]
    band = ff_vals[ff_idx].split('_')[-1]
    group = ff_groups[ff_idx]
    
    #if field not in ['MIRI5']: continue
    #if band not in ['F770W']: continue
    
    # Create an association 
    asn = dict( asn_from_list.asn_from_list(group, rule=Asn_Lv3Image, 
                product_name='l3_results', asn_type="image3") )
    # Initialize stage 3 pipeline
    # In the following, we use ".copy()" as input to avoid the input being changed 
    img3 = Image3Pipeline()
    # Load the data models
    dms = datamodels.open(asn)

    # DO a strict CR rejection for short wavelength
    if ('560' in ff_vals[ff_idx]) or ('770' in ff_vals[ff_idx]):
        cr_sigclip=0
    else: 
        cr_sigclip=1
    tkreg_res = run_tkreg(dms, img3, cr_sigclip=cr_sigclip, run2=True)

    # match bkg
    img3.skymatch.subtract = True
    img3.skymatch.skymethod = 'local'
    res = img3.skymatch.run(tkreg_res.copy())

    # Reject cosmic-ray 
    res = img3.outlier_detection.run(res)
        
    # Manually mask the latent for June-28 F2100W observations
    for exp_idx in range(len(res)):
        # Check if the exposure is June-28
        if (res[exp_idx].meta.time.heliocentric_expmid>59758.3) and \
           (res[exp_idx].meta.time.heliocentric_expmid<59758.4):
            if (band!='F2100W') or (field!='MIRI2'): 
                raise Exception('Latent field/band not matched?!')
            print("latent mask applied!!")
            # Apply the mask
            mask_map = fits.getdata('../data/persistence/latent_mask.fits')
            mask = mask_map!=0
            res[exp_idx].data[mask] = 0
            res[exp_idx].dq[mask] = 11 # the flag is rather some arbitrary bad 
            res[exp_idx].err[mask] = np.inf
            res[exp_idx].var_rnoise[mask] = np.inf
    
    # First resample to get the coordinate transformation
    img3.resample.rotation = -49.7
    img3.resample.pixel_scale = 0.09
    img3.resample.output_shape = out_pars[field][0]
    tg_ra, tg_dec = 214.825, 52.825
    img3.resample.crval = tg_ra, tg_dec
    
    # Use the pre-determined tangential x,y
    img3.resample.crpix = tg_xys[ ff_vals[ff_idx][:5] ]
    res = img3.resample(res)
    # Output 
    output_fname = output_dir.replace('reduced', 'release/%s/' %version) + \
                'ceers_' + ff_vals[ff_idx].lower() + '_i2d.fits'
    res.save(output_fname)
    
# Clean up temporary files from intermediate steps
os.system('rm *.fits')

# Produce the RMS map
stage3_files = sorted(glob(output_dir.replace('reduced', 'release/%s/' %version) + '*i2d.fits'))
# Array to save RMS scaling factor
rms_facs = []
for file in stage3_files:
    rms_facs.append( write_rms_map(file, input_type='wht') )

2023-04-16 16:46:18,978 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2023-04-16 16:46:18,980 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2023-04-16 16:46:18,982 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2023-04-16 16:46:18,984 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2023-04-16 16:46:18,985 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2023-04-16 16:46:18,987 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2023-04-16 16:46:18,989 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.
2023-04-16 16:46:19,262 - stpipe.Image3Pipeline.resample - INFO - Step resample running with args ({'asn_type': 'None', 'asn_rule': 'Asn_Lv3Image', 'version_id': None, 'code_version': '1.10.2.dev7+g8fb5bd7d', 'degraded_status': 'No known degraded exposures in association.', 'program': 'noprog

MIRI1 does not exist; skip...
MIRI2 does not exist; skip...
MIRI3 does not exist; skip...
MIRI6 does not exist; skip...


2023-04-16 16:46:19,264 - stpipe.Image3Pipeline.resample - INFO - Step resample parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'pixfrac': 1.0, 'kernel': 'square', 'fillval': 'INDEF', 'weight_type': 'ivm', 'output_shape': [1800, 2000], 'crpix': None, 'crval': [214.825, 52.825], 'rotation': -49.7, 'pixel_scale_ratio': 1.0, 'pixel_scale': 0.09, 'output_wcs': '', 'single': False, 'blendheaders': True, 'allowed_memory': None, 'in_memory': True}
2023-04-16 16:46:19,973 - stpipe.Image3Pipeline.resample - INFO - Using drizpars reference file: /Users/users/gyang/gyang/crds_cache/jwst_ops/references/jwst/miri/jwst_miri_drizpars_0001.fits
2023-04-16 16:46:19,999 - stpipe.Image3Pipeline.resample - INFO - Driz parameter kernel: square
2023-04-16 16:46:20,000 - stpipe.Image3Pipeline

2023-04-16 16:46:47,066 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:46:48,494 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:46:49,761 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:46:51,031 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:46:52,323 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:46:53,596 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:46:55,025 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:46:56,347 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:46:56,791 - stpipe.Image3Pipeline.resample - INFO - Resampling var_rnoise
2023-04-16 16:46:57,745 - stpipe.Image3Pipeline.resample - INFO 

2023-04-16 16:47:36,753 - stpipe.Image3Pipeline.resample - INFO - Blending metadata for l3_results
2023-04-16 16:47:38,010 - stpipe.Image3Pipeline.resample - INFO - Resampling science data
2023-04-16 16:47:38,770 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:47:39,931 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:47:40,991 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:47:41,351 - stpipe.Image3Pipeline.resample - INFO - Resampling var_rnoise
2023-04-16 16:47:42,162 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:47:43,378 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:47:44,559 - stpipe.Image3Pipeline.resample - INFO - Drizzling (1024, 1032) --> (2000, 1800)
2023-04-16 16:47:45,011 - stpipe.Image3Pipeline.resample - INFO - Resampling var_poisso

...

2023-04-16 17:04:27,835 - stpipe.Image3Pipeline.tweakreg - WARNING -         Use JWSTWCSCorrector instead.
2023-04-16 17:04:27,835 - stpipe.Image3Pipeline.tweakreg - WARNING -   im = JWSTgWCS(
2023-04-16 17:04:27,836 - stpipe.Image3Pipeline.tweakreg - WARNING - 
2023-04-16 17:04:27,861 - stpipe.Image3Pipeline.tweakreg - INFO - * Images in GROUP 'jw01345012001_05201_00001_mirimage_cal_stprm_bgsub':
2023-04-16 17:04:27,861 - stpipe.Image3Pipeline.tweakreg - INFO -      jw01345012001_05201_00001_mirimage_cal_stprm_bgsub
2023-04-16 17:04:27,887 - stpipe.Image3Pipeline.tweakreg - INFO - * Images in GROUP 'jw01345012001_03201_00001_mirimage_cal_stprm_bgsub':
2023-04-16 17:04:27,888 - stpipe.Image3Pipeline.tweakreg - INFO -      jw01345012001_03201_00001_mirimage_cal_stprm_bgsub
2023-04-16 17:04:27,917 - stpipe.Image3Pipeline.tweakreg - INFO - * Images in GROUP 'jw01345012001_05201_00002_mirimage_cal_stprm_bgsub':
2023-04-16 17:04:27,918 - stpipe.Image3Pipeline.tweakreg - INFO -      jw013450

2023-04-16 17:04:29,641 - stpipe.Image3Pipeline.tweakreg - INFO -  
2023-04-16 17:04:29,844 - stpipe.Image3Pipeline.tweakreg - INFO - Step tweakreg done
2023-04-16 17:04:30,148 - stpipe.Image3Pipeline.tweakreg - INFO - Step tweakreg running with args (<ModelContainer>,).
2023-04-16 17:04:30,150 - stpipe.Image3Pipeline.tweakreg - INFO - Step tweakreg parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'tweakreg', 'search_output_file': True, 'input_dir': '', 'save_catalogs': False, 'catalog_format': 'ecsv', 'kernel_fwhm': 2.5, 'snr_threshold': 10.0, 'brightest': 200, 'peakmax': None, 'enforce_user_order': False, 'expand_refcat': False, 'minobj': 15, 'searchrad': 1, 'use2dhist': True, 'separation': 0.01, 'tolerance': 0.5, 'xoffset': 0.0, 'yoffset': 0.0, 'fitgeometry': 'rshift', 'nclip': 3, 'sigma': 3.0, 'align_to_gaia': False, 

2023-04-16 17:04:31,152 - stpipe.Image3Pipeline.tweakreg - WARNING - 
2023-04-16 17:04:31,152 - stpipe.Image3Pipeline.tweakreg - INFO -  
2023-04-16 17:04:31,153 - stpipe.Image3Pipeline.tweakreg - INFO - ***** tweakwcs.imalign.align_wcs() started on 2023-04-16 17:04:31.152541
2023-04-16 17:04:31,153 - stpipe.Image3Pipeline.tweakreg - INFO -       Version 0.8.2
2023-04-16 17:04:31,153 - stpipe.Image3Pipeline.tweakreg - INFO -  
2023-04-16 17:04:31,625 - stpipe.Image3Pipeline.tweakreg - INFO - Aligning image catalog 'GROUP ID: 876543' to the reference catalog.
2023-04-16 17:04:31,834 - stpipe.Image3Pipeline.tweakreg - INFO - Matching sources from 'jw01345012001_0' catalog with sources from the reference 'Unnamed' catalog.
2023-04-16 17:04:31,837 - stpipe.Image3Pipeline.tweakreg - INFO - Computing initial guess for X and Y shifts...
2023-04-16 17:04:31,875 - stpipe.Image3Pipeline.tweakreg - INFO - Found initial X and Y shifts of 0.08758, 0.07071 (arcsec) with significance of 51.24 and 119

2023-04-16 17:04:44,486 - stpipe.Image3Pipeline.outlier_detection - WARNING - 
2023-04-16 17:04:45,537 - stpipe.Image3Pipeline.outlier_detection - INFO - Blotting (1024, 1032) <-- (1045, 1245)
2023-04-16 17:04:46,110 - stpipe.Image3Pipeline.outlier_detection - WARNING - /net/dataserver3/data/users/gyang/software/jwst/jwst/outlier_detection/outlier_detection.py:337: RuntimeWarning: invalid value encountered in multiply
2023-04-16 17:04:46,111 - stpipe.Image3Pipeline.outlier_detection - WARNING -   blotted_median.err *= 0.0  # None
2023-04-16 17:04:46,111 - stpipe.Image3Pipeline.outlier_detection - WARNING - 
2023-04-16 17:04:47,422 - stpipe.Image3Pipeline.outlier_detection - INFO - Blotting (1024, 1032) <-- (1045, 1245)
2023-04-16 17:04:47,876 - stpipe.Image3Pipeline.outlier_detection - INFO - Flagging outliers
2023-04-16 17:04:48,388 - stpipe.Image3Pipeline.outlier_detection - WARNING - /net/dataserver3/data/users/gyang/software/jwst/jwst/outlier_detection/outlier_detection.py:442: Run

2023-04-16 17:05:14,920 - stpipe - WARNING - /net/dataserver3/data/users/gyang/sterm_bkup/ownCloud/notebook/ceers/reduction/astroRMS.py:476: RuntimeWarning: divide by zero encountered in divide
2023-04-16 17:05:14,921 - stpipe - WARNING -   weight_data = np.divide(1.0, weight_data)
2023-04-16 17:05:14,922 - stpipe - WARNING - 
2023-04-16 17:05:16,679 - stpipe - WARNING - /net/dataserver3/data/users/gyang/sterm_bkup/ownCloud/notebook/ceers/reduction/astroRMS.py:401: UserWarning: Ran out of good pixels when placing RMS calculation regions for file [[0. 0. 0. ... 0. 0. 0.]
2023-04-16 17:05:16,681 - stpipe - WARNING -  [0. 0. 0. ... 0. 0. 0.]
2023-04-16 17:05:16,681 - stpipe - WARNING -  [0. 0. 0. ... 0. 0. 0.]
2023-04-16 17:05:16,682 - stpipe - WARNING -  ...
2023-04-16 17:05:16,682 - stpipe - WARNING -  [0. 0. 0. ... 0. 0. 0.]
2023-04-16 17:05:16,683 - stpipe - WARNING -  [0. 0. 0. ... 0. 0. 0.]
2023-04-16 17:05:16,683 - stpipe - WARNING -  [0. 0. 0. ... 0. 0. 0.]]. Only 9 boxes selected

In [ ]:
# ZP calculation 

# Conversion factor from MJy/sr to uJy/arcsec^2
MJSR_TO_UJA2 = (u.megajansky/u.steradian).to(u.microjansky/u.arcsecond/u.arcsecond)
#pix_scale = 3.08076303795716E-05*3600
pix_scale = 0.09
pixelarea_arcsecsq = pix_scale**2
conv_fac = MJSR_TO_UJA2*pixelarea_arcsecsq

zp = 23.9-2.5*np.log10(conv_fac)
print("ZP = %.5f mag" %zp)

# Find detector edge based on error map
def find_edge(err_img, shift=-2):
    region = ~np.isinf(err_img)
    nregion = ~region
    edgex = (region ^ np.roll(nregion,shift=shift,axis=0))
    edgey = (region ^ np.roll(nregion,shift=shift,axis=1))
    all_edge = np.where(~edgex | ~edgey)
    return all_edge

In [ ]:
for field in ['miri1', 'miri2']:
    for flt in ['f770w', 'f1000w', 'f1280w', 'f1500w', 'f1800w', 'f2100w']:
        print(field, flt, fits.getheader('../data/release/latest/ceers_%s_%s_i2d.fits' %(field, flt))['EFFEXPTM'])

for field in ['miri3', 'miri6']:
    for flt in ['f560w', 'f770w']:
        print(field, flt, fits.getheader('../data/release/latest/ceers_%s_%s_i2d.fits' %(field, flt))['EFFEXPTM'])        
            
# The exposures have the jump issue 
for i in [1, 2]:
    exp_tot = 0
    for j in [1, 2, 3]:
        jump_file = '../data/reduced/jw0134500%d001_14101_0000%d_mirimage_rate.fits' %(i,j)
        hd = fits.getheader( jump_file )
        exp_tot += hd['EFFEXPTM']
    print('jump exposure: ', hd['TARGPROP'], '%.3f' %exp_tot)
    
# The persistence exposures
exp_tot = 0
for j in [1, 2, 3]:
    file = '../data/reduced/jw01345052001_02101_0000%d_mirimage_cal.fits' %(j)
    #file = '../data/reduced/jw01345002001_12101_0000%d_mirimage_cal.fits' %(j)
    hd = fits.getheader( file )
    exp_tot += hd['EFFEXPTM']
print('latent exposure: ', hd['TARGPROP'], '%.3f' %exp_tot)

In [ ]:
# Below is the parallel version of stage 3
# it may cause dead kenel

'''
# Define the function to parallel
def do_ff(ff_idx):
    group = ff_groups[ff_idx]
    
    # Create an association 
    asn = dict( asn_from_list.asn_from_list(group, rule=Asn_Lv3Image, 
                product_name='l3_results', asn_type="image3") )
    # Initialize stage 3 pipeline
    # In the following, we use ".copy()" as input to avoid the input being changed 
    img3 = Image3Pipeline()
    # Load the data models
    dms = datamodels.open(asn)

    # DO a strict CR rejection for short wavelength
    if ('560' in ff_vals[ff_idx]) or ('770' in ff_vals[ff_idx]):
        cr_sigclip=0
    else: 
        cr_sigclip=1
    tkreg_res = run_tkreg(dms, img3, cr_sigclip=cr_sigclip, run2=True)

    # match bkg
    img3.skymatch.subtract = True
    img3.skymatch.skymethod = 'local'
    res = img3.skymatch.run(tkreg_res.copy())

    # Reject cosmic-ray 
    res = img3.outlier_detection.run(res)
        
    # Manually mask the latent for June-28 F2100W observations
    for exp_idx in range(len(res)):
        # Check if the exposure is June-28
        if (res[exp_idx].meta.time.heliocentric_expmid>59758.3) and \
           (res[exp_idx].meta.time.heliocentric_expmid<59758.4):
            print("laten mask applied!!")
            # Apply the mask
            mask_map = fits.getdata('../data/persistence/latent_mask.fits')
            mask = mask_map!=0
            res[exp_idx].data[mask] = 0
            res[exp_idx].dq[mask] = 11 # the flag is rather some arbitrary bad 
            res[exp_idx].err[mask] = np.inf
            res[exp_idx].var_rnoise[mask] = np.inf
    
    # First resample to get the coordinate transformation
    img3.resample.rotation = -49.7
    img3.resample.pixel_scale = 0.09
    img3.resample.output_shape = 1500, 1500
    tg_ra, tg_dec = 214.825, 52.825
    img3.resample.crval = tg_ra, tg_dec
    
    # Run a temporary resample to roughly calculate the TG point
    #temp_res = img3.resample(res.copy())    
    #tg_x, tg_y = temp_res.get_fits_wcs().world_to_pixel( SkyCoord(tg_ra*u.degree, tg_dec*u.degree, frame='icrs') )
    # Round to 0.5 to align with HST 
    #tg_x, tg_y = np.round(tg_x)+60.5, np.round(tg_y)+60.5
    # Resample again to get the TG point correct
    
    # Use the pre-determined tangential x,y
    img3.resample.crpix = tg_xys[ ff_vals[ff_idx][:5] ]
    res = img3.resample(res)
    # Output 
    output_fname = output_dir.replace('reduced', 'release/%s/' %version) + \
                'ceers_' + ff_vals[ff_idx].lower() + '_i2d.fits'
    res.save(output_fname)

# Decide number of cores to use
Nc = min(multiprocessing.cpu_count()//20, len(ff_vals))
# Run
with multiprocessing.Pool(processes=Nc) as pool_obj:
    pool_obj.map(do_ff, range(len(ff_vals)))


# Remove background 
stage2_files = sorted(glob(output_dir+'*stprm.fits'))
# Extract filter info for each file 
flt_groups, flt_vals = group_imgs(stage2_files, 'band')
# subtract the background group by group
for flt, files in zip(flt_vals, flt_groups):
    print(flt+':')
    # Define the parallel function
    def do_f0(f0): 
        super_bg(f0, files)
        return
    # Decide number of cores to use
    Nc = min(multiprocessing.cpu_count()//4, len(files))
    # Run
    with multiprocessing.Pool(processes=Nc) as pool_obj:
        pool_obj.map(do_f0, files)
'''